# Perceptron et réseaux de neurones

Documentation des librairies utilisées dans ce tp:
- scikit-learn: [https://scikit-learn.org/stable/index.html](https://scikit-learn.org/stable/index.html)
- matplotlib: [https://matplotlib.org/](https://matplotlib.org/)
- numpy: [https://numpy.org/](https://numpy.org/)

Instructions d'installation dans un environnement conda, si vous voulez utiliser votre propre ordinateur (inutile de suivre ces instructions sur les machines de la salle info).

- Créer un nouvel environnement, appelé `sd3` (vous avez besoin d'avoir anaconda ou miniconda installé)
```
conda create -n sd3 python=3.9
```
- Activer l'environnement
```
conda activate sd3
```
- Installer les librairies requises
```
conda install -c conda-forge jupyterlab matplotlib scikit-learn
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.datasets import load_digits, make_classification, fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

### Classer des chiffres

- Charger l'ensemble d'exemples `digits` de scikit-learn à l'aide de la fonction `load_digits`.
- Placer les données dans un tableau `X` et les étiquettes dans un vecteur `Y`.
- Séparer les données en un ensemble d'entrainement et un ensemble de test (40% des données) à l'aide de la fonction `train_test_split`.

On pourra utiliser la fonction suivante pour observer des exemples des données qu'on vient de charger:

In [ ]:
def plot_examples(data, labels):  # données, étiquettes
    _, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
    nb_pxl = len(data[0].flatten())
    side_len = int(np.sqrt(nb_pxl))  # taille du côté de l'image (supposée carrée)
    for ax, image, label in zip(axes, data, labels):
        ax.set_axis_off()
        image = image.reshape((side_len, side_len))
        ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
        ax.set_title("Label: %i" % label)

1) Que sont les données ? Et leurs étiquettes ?

2) Pourquoi est-ce qu'on coupe les exemples en un ensemble d'entrainement et un ensemble de test ?

3) Combien de classes (= étiquettes différentes) dans cet ensemble d'exemples ?

4) Est-ce que ce nombre de classes est un problème pour la classification avec un classeur des plus proches voisins ? Et pour un perceptron ?
5) Si c'est un problème, proposer des solutions.

6) Construire un classeur k-NN avec k=5 et calculer son taux d'erreur sur l'ensemble de test. Rappel: utiliser `KNeighborsClassifier`.

La fonction suivante permet d'afficher une matrice de confusion pour un ensemble d'étiquettes `Y` et de prédictions `pred`. La case de la ligne $i$ et colonne $j$ de la matrice représente le nombre de fois où une donnée d'étiquette $i$ (dans `Y`) correspond à une prédiction $j$ dans `pred`.

In [ ]:
def plot_confusion_matrix(Y, pred):
    disp = ConfusionMatrixDisplay.from_predictions(Y, pred)
    disp.figure_.suptitle("Confusion Matrix")
    plt.show()

7) Afficher la matrice de confusion pour le k-NN entrainé précédemment. 

### Le perceptron

Le perceptron est un classeur constitué d'un seul neurone, qui permet d'effectuer une classification binaire.

La classe `Perceptron` de scikit-learn permet aussi de faire de la classification à $K>2$ classes : $K$ perceptrons différents sont construits, et le perceptron numéro $k$ a pour tâche de classer les données de manière binaire: il prédit soit "classe $k$" soit "autre classe". La prédiction finale est la classe $k \in \{1, \ldots, K\}$ pour laquelle le perceptron $k$ a la sortie la plus haute pour sa classe.

Exemple de création d'un perceptron:

In [ ]:
per = Perceptron(tol=0.001, max_iter=1000, shuffle=True, eta0=1.0, random_state=0)

`tol` est un paramètre qui définit le critère d'arrêt de l'apprentissage: l'algorithme arrête de changer les poids (avec une descente de gradient) si la performance sur les données d'apprentissage ne s'améliore pas plus que `tol`.

`max_iter` dicte le nombre maximal d'époques lors de la descente de gradient.

`eta0` est le paramètre qui règle la taille du pas de gradient effectué à chaque étape de l'apprentissage.

8) Entrainer le classeur et obtenir son score sur les données d'entrainement et de test (utiliser `per.score()`). Le taux d'erreur est $1 - \text{score}$. Comparer au k-NN
9) Afficher la matrice de confusion du classeur entrainé

10) La fonction `partial_fit` permet de faire une itération de la règle d'entrainement (utiliser chaque donnée d'entrainement une fois pour changer les poids). A l'aide de cette fonction, construire un graphe du score du perceptron sur les données d'entrainement et de test au cours de l'apprentissage.
11) Qu'observez vous sur ce graphe concernant le score d'apprentissage ? Et le score de test ?

### Le perceptron multi-couches

Le perceptron multi-couches, ou réseau de neurones complètement connecté, est constitué de plusieurs couches de neurones.

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(15,10,), activation="relu",
    solver="sgd", learning_rate="constant", learning_rate_init=0.001,
    max_iter=2000, verbose=False)

`hidden\_layer\_sizes=(15,10,)` indique le nombre de neurones dans chaque couche cachée du réseau de neurones.

`activation` décide la fonction d'activation de chaque neurone.

`solver` indique la méthode d'apprentissage des poids. "sgd" = descente de gradient stochastique.

`learning_rate_init` indique la taille de chaque pas de gradient dans la méthode des gradients stochastiques.

12) Entrainer le classeur et obtenir son score sur les données d'entrainement et de test.

13) Afficher la matrice de confusion du classeur. Comparer avec le perceptron.

14) Tracer une courbe des erreurs d'entrainement et de test au fil de l'apprentissage, comme effectué pour le perceptron plus haut.

15) Essayer plusieurs architectures (valeurs de `hidden_layer_sizes`).

16) Faire varier `learning_rate_init` et observer l'effet sur l'apprentissage.

17) Changer le paramètre `learning_rate`: essayer les valeurs `adaptive` et `invscaling`. Chercher dans la documentation ce que veulent dire ces paramètres.

### Données MNIST

On va utiliser ici le jeu de données MNIST, qui contient des images de chiffres écrits à la main, représentés par des images en nuances de gris de 28x28 pixels.
On peut obtenir ces données grâce au code écrit dans la cellule suivante.
L'obtention du jeu de données peut prendre du temps: il est conseillé de n'exécuter ce code qu'une fois.

In [ ]:
mnist = fetch_openml("mnist_784", version=1, as_frame=False)

- Combien de données sont présentes dans mnist ? Combien de classes ? Afficher quelques exemples à l'aide de la fonction `plot_examples` définie plus haut.
- Séparer l'ensemble de données en données d'entrainement et de test. Attention: l'ensemble de données est très gros. Si vous prenez toutes les données, l'apprentissage des classeurs peut être très lent. Utilisez les paramètres `train_size=..., test_size=...` de la fonction `train_test_split` pour obtenir des ensembles d'entrainement et de test de la taille voulue.
- Entrainer un perceptron sur ces données. Quel est son score ?
- Entrainer des MLP divers sur ces données. Est-il possible de faire mieux que le perceptron ?
- Le code suivant permet de recentrer les valeurs des pixels autour de zéro. Est-ce que l'apprentissage d'un MLP est plus efficace après ce changement ?

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)